In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModel, AutoImageProcessor

model_name = "DAMO-NLP-SG/VideoLLaMA3-2B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
video_path = "put your video path here"
image_path = "put your image path here"
question = "Describe this video in detail."

2025-02-17 21:04:50.222902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739826290.231135  182475 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739826290.234848  182475 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
# Video conversation
video_path = "/home/ubuntu/sample.mp4"
#image_path = "/home/ubuntu/45s.png"
image_path = "/home/ubuntu/graduation.png"


#sample question
#question = "What happens at time: 10 seconds?"


#image based QA
#image_caption = "A modern educational or institutional building with a green lawn, featuring a white facade and large glass windows under a clear blue sky. A police emblem watermark is visible in the upper right corner"
question = "What's his name"


conversation = [
    {"role": "system", "content": "You are a helpful and informed assistant."},
    {
        "role": "user",
        "content": [
            {"type": "video", "video": {"video_path": video_path, "fps": 1, "max_frames": 1000, "start_time":50.0, "end_time":70.0}},
            {"type": "image", "image": {"image_path" : image_path}},
            {"type": "text", "text": question},
        ]
    },
]

inputs = processor(conversation=conversation, return_tensors="pt")
inputs = {k: v.cuda() if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
if "pixel_values" in inputs:
    inputs["pixel_values"] = inputs["pixel_values"].to(torch.bfloat16)
output_ids = model.generate(**inputs, max_new_tokens=256)
response = processor.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
print(response)


The man's name is Joe Jaramazovic.


In [4]:
inputs["input_ids"].shape
token_length = inputs["input_ids"].shape[1]
print(f"Tokenized sequence length: {token_length}")



Tokenized sequence length: 7488


In [5]:
inputs

{'input_ids': tensor([[151644,   8948,    198,  ...,    829, 151645,    198]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
 'pixel_values': tensor([[-0.1455, -0.1455, -0.1377,  ..., -0.4668, -0.4746, -0.4824],
         [-0.0747, -0.0747, -0.0747,  ..., -0.2793, -0.2637, -0.2793],
         [-0.3730, -0.3809, -0.4043,  ..., -0.3027, -0.3105, -0.3105],
         ...,
         [-0.6719, -0.6562, -0.6484,  ..., -0.6562, -0.6484, -0.6484],
         [-0.6172, -0.6016, -0.5938,  ..., -0.6250, -0.6328, -0.6406],
         [-0.6719, -0.6719, -0.6719,  ..., -0.6562, -0.6562, -0.6562]],
        device='cuda:0', dtype=torch.bfloat16),
 'grid_sizes': tensor([[20, 26, 46],
         [ 1, 45, 29]], device='cuda:0'),
 'merge_sizes': tensor([2, 1], device='cuda:0'),
 'modals': ['video', 'image']}

In [6]:
a = model.forward(**inputs, return_dict=True, use_cache=True)

In [7]:
a.past_key_values

DynamicCache()

In [8]:
a

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 1.6094,  1.1562,  2.6875,  ..., -1.2188, -1.2188, -1.2188],
         [ 6.9062,  4.1562,  6.9062,  ..., -0.6875, -0.6875, -0.6875],
         [ 4.7812,  6.6250,  9.0000,  ..., -3.6562, -3.6562, -3.6562],
         ...,
         [12.1250,  8.4375,  6.2188,  ...,  0.4922,  0.4922,  0.4922],
         [ 7.0000,  3.6719,  5.3125,  ..., -3.2656, -3.2656, -3.2656],
         [ 6.0312,  9.6875,  6.4062,  ..., -2.7500, -2.7500, -2.7500]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(), hidden_states=None, attentions=None)

In [9]:
a.logits.shape

torch.Size([1, 6665, 151936])

In [11]:
a.past_key_values[0][0].shape

torch.Size([1, 2, 6665, 128])